In [5]:
from langgraph.graph import StateGraph, END
from backend.rag.retriever import retrieve_resources
from backend.prompt.prompt import UPSKILL_RECOMMENDATION_PROMPT
from backend.llm.gemini import gemini_chat
from typing import TypedDict
import json
import re


In [6]:
class UpskillAgentState(TypedDict):
    offer_components: dict
    acceptance_prediction: dict
    onboarding_resources: dict

In [7]:
def clean_llm_json_response(raw_response: str) -> dict:
    try:
        # Remove ```json or ``` if present
        cleaned = re.sub(r"```(?:json)?", "", raw_response).strip("`\n ")
        return json.loads(cleaned)
    except Exception as e:
        return {"error": f"Failed to parse LLM response: {str(e)}", "raw": raw_response}


def fetch_resources_with_rag(state: UpskillAgentState) -> UpskillAgentState:
    role = state["offer_components"].get("job_title", "")
    company = state["offer_components"].get("company", "")
    tools = state["offer_components"].get("tech_stack", "")
    
    resources = retrieve_resources(role, company, tools)
    return {**state, "retrieved_resources": resources}

def recommend_upskill_resources(state: UpskillAgentState) -> UpskillAgentState:
    "======================"
    prompt = UPSKILL_RECOMMENDATION_PROMPT.format(
        offer=state["offer_components"],
        resources=state["retrieved_resources"]
    )
    response = gemini_chat(prompt)
    
    print("response", response)

    try:
        structured = clean_llm_json_response(response)
    except Exception as e:
        print("Error in upskill")
        structured = {"error": str(e)}
    
    return {**state, "onboarding_resources": structured}


In [8]:
builder = StateGraph(UpskillAgentState)
builder.add_node("fetch_resources", fetch_resources_with_rag)
builder.add_node("llm_recommendation", recommend_upskill_resources)

builder.set_entry_point("fetch_resources")
builder.add_edge("fetch_resources", "llm_recommendation")
builder.add_edge("llm_recommendation", END)

app = builder.compile()

In [9]:
result = app.invoke({"offer_components": "```python\njob_offer_details = {\n    'ctc': '₹ 6-9 LPA',\n    'bonuses': None,  # Not mentioned in the text\n    'job_title': 'Software Developer',\n    'responsibilities': [\n        'Develop and maintain web applications using modern programming languages and frameworks.',\n        'Collaborate with cross-functional teams to design, define, and deliver new features.',\n        'Write clean, reusable, and efficient code.',\n        'Implement and maintain robust security and data protection measures.',\n        'Optimize applications for performance and scalability.',\n        'Troubleshoot and resolve development and production issues.',\n        'Stay updated with emerging technologies and industry trends.'\n    ],\n    'location': 'Coimbatore/Bangalore',\n    'bond_duration': None,  # Not mentioned in the text\n    'benefits': None #Not explicitly listed, but implied by the description of SNSiHub's offerings.\n}\n\nprint(job_offer_details)\n```", "acceptance_prediction": '```json\n{\n  "probability": 40,\n  "reasons": [\n    "CTC is lower than candidate\'s expected salary (10 LPA)",\n    "Mismatch in work location preference (candidate prefers Coimbatore, offer includes Bangalore)"\n  ]\n}\n```'})

AttributeError: 'str' object has no attribute 'get'